In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt
import time
from datetime import timedelta
import pandas_ta as ta

from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from functools import reduce



In [2]:
def get_data(ticker, year,month,day):
    start = dt.datetime(1999,1,1)
    end = dt.datetime.now()
     

    df = yf.download(ticker,start,end)
    
    return df

In [3]:
df = get_data('SPY',1999,1,1)

[*********************100%***********************]  1 of 1 completed


In [4]:
df 

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-01-04,123.375000,125.218750,121.718750,123.031250,79.169907,9450400
1999-01-05,122.937500,124.875000,122.937500,124.437500,80.074829,8031000
1999-01-06,125.812500,127.750000,125.750000,127.437500,82.005302,7737700
1999-01-07,126.375000,127.218750,125.781250,126.812500,81.603127,5504900
1999-01-08,128.187500,128.500000,125.968750,127.750000,82.206390,6224400
...,...,...,...,...,...,...
2023-08-18,433.369995,437.570007,433.010010,436.500000,436.500000,98758400
2023-08-21,437.549988,440.109985,435.320007,439.339996,439.339996,68719000
2023-08-22,441.179993,441.179993,437.570007,438.149994,438.149994,65062900


In [4]:
# add features to model
df['Range'] = abs(df['High']-df['Low'])
df['RSI 14'] = ta.rsi(df['Close'], 14)
df['SMA 20'] = ta.sma(df['Close'], 20)
df['SMA 50'] = ta.sma(df['Close'], 50)
df['SMA 200'] = ta.sma(df['Close'], 200)
df['EMA 12'] = ta.ema(df['Close'], 12)
df['EMA 26'] = ta.ema(df['Close'], 26)
df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'] )

# create 'Tomorrow' column
# it's the 'Close' value from the previous day
# will be used to create a target
df['Tomorrow'] = df['Close'].shift(-1)
df['Target'] = (df['Tomorrow'] > df['Close']).astype(int)
df = df.loc['2000-01-01':]
df.drop('Adj Close', axis=1, inplace=True)
df.drop(df.tail(1).index, inplace=True)

C:\Users\13148\AppData\Local\Temp\ipykernel_20708\3650632616.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('Adj Close', axis=1, inplace=True)
C:\Users\13148\AppData\Local\Temp\ipykernel_20708\3650632616.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.tail(1).index, inplace=True)


In [5]:
df

,Open,High,Low,Close,Volume,Range,RSI 14,SMA 20,SMA 50,SMA 200,EMA 12,EMA 26,ATR,Tomorrow,Target
Date,,,,,,,,,,,,,,,
2000-01-03,148.250000,148.250000,143.875000,145.437500,8164300,4.375000,60.123229,143.598437,140.252500,134.929844,145.100225,143.284849,1.978058,139.750000,0
2000-01-04,143.531250,144.062500,139.640625,139.750000,8089800,4.421875,39.912317,143.446875,140.445625,134.978906,144.277113,143.023008,2.250830,140.000000,1
2000-01-05,139.937500,141.531250,137.250000,140.000000,12177900,4.281250,40.853504,143.365625,140.656875,135.047969,143.619096,142.799082,2.395860,137.750000,0
2000-01-06,139.625000,141.500000,137.750000,137.750000,6227200,3.750000,35.468775,143.217187,140.855625,135.102188,142.716158,142.425076,2.492585,145.750000,1
2000-01-07,140.312500,145.750000,140.062500,145.750000,8066500,5.687500,57.113342,143.434375,141.168125,135.183437,143.182903,142.671366,2.885971,146.250000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-18,433.369995,437.570007,433.010010,436.500000,98758400,4.559998,35.283750,448.740503,444.149401,412.000000,443.726081,445.381265,4.519322,439.339996,1
2023-08-21,437.549988,440.109985,435.320007,439.339996,68719000,4.789978,40.887683,447.997502,444.353600,412.274100,443.051299,444.933763,4.538655,438.149994,0
2023-08-22,441.179993,441.179993,437.570007,438.149994,65062900,3.609985,39.350095,447.133002,444.518600,412.590500,442.297252,444.431262,4.472321,443.029999,1


In [6]:
# loop through dataframe and add all features to a list
# removes 'Tomorrow' and 'Target' column from the list 
def get_features(dataframe):
    features = []
    for column in dataframe:
        features.append(column)
    
    features.pop(-1)
    features.pop(-1)
    
    return features


In [9]:
features = get_features(df)

In [7]:
def fit_train_score(df, n_est, min_split):    
    # n_estimators = number of decision trees
    # min_samples_split = higher it is set, the less accurtate it is, the less it will overfit
    model = RandomForestClassifier(n_estimators=n_est, min_samples_split=min_split, random_state=1)

    train = df.iloc[:-100]
    test = df.iloc[-100:]

    # fit and train model
    model.fit(train[features], train['Target'])
    predictions = model.predict(test[features])
    #predictions_series = pd.Series(predictions, index=test.index)

    # precision of training data
    predictions_training = model.predict(train[features])
    
    test_dict = {
        'n_estimators':n_est,
        'min_samples_split':min_split,
        'training precision':precision_score(train['Target'], predictions_training),
        'testing precision':precision_score(test['Target'], predictions)
                }
    
    return test_dict


In [10]:
fit_train_score(df,200,750)

{'n_estimators': 200,
 'min_samples_split': 750,
 'training precision': 0.5778996546821044,
 'testing precision': 0.5408163265306123}

In [11]:
fit_train_score(df,300,400)

{'n_estimators': 300,
 'min_samples_split': 400,
 'training precision': 0.6020517204530883,
 'testing precision': 0.5353535353535354}

In [17]:
# loop through possible n_estimator values and min_sample_split values
start_time = time.time()

test_list = []
for n in range(200,301,50):

    for m in range(500,801,50):

        result_dict = fit_train_score(df,n,m)
        test_list.append(result_dict)
        print("--- %s minutes ---" % ((time.time() - start_time)/60))
        print(result_dict)
        print("")

results_df = pd.DataFrame(test_list)
print("--- %s minutes ---" % ((time.time() - start_time)/60))

--- 0.2163983702659607 minutes ---
{'n_estimators': 200, 'min_samples_split': 500, 'training precision': 0.5914494264859228, 'testing precision': 0.5408163265306123}

--- 0.42201440731684364 minutes ---
{'n_estimators': 200, 'min_samples_split': 550, 'training precision': 0.5873806558738066, 'testing precision': 0.5306122448979592}

--- 0.6243352929751078 minutes ---
{'n_estimators': 200, 'min_samples_split': 600, 'training precision': 0.5851195383347073, 'testing precision': 0.5306122448979592}

--- 0.8213376641273499 minutes ---
{'n_estimators': 200, 'min_samples_split': 650, 'training precision': 0.583607230895645, 'testing precision': 0.5353535353535354}

--- 1.020215348402659 minutes ---
{'n_estimators': 200, 'min_samples_split': 700, 'training precision': 0.5806781045751634, 'testing precision': 0.5306122448979592}

--- 1.2249446352322897 minutes ---
{'n_estimators': 200, 'min_samples_split': 750, 'training precision': 0.5771648752788481, 'testing precision': 0.5408163265306123}


In [18]:
results_df['testing delta'] = abs(results_df['testing precision'] - results_df['training precision'])*100
results_df_precision_sorted = results_df.copy().sort_values(by='testing precision', ascending=False)

In [20]:
results_df_precision_sorted.head(20)

,n_estimators,min_samples_split,training precision,testing precision,testing delta
7,250,500,0.590672,0.545455,4.521795
14,300,500,0.588898,0.545455,4.344362
0,200,500,0.591449,0.540816,5.063310
5,200,750,0.577165,0.540816,3.634855
17,300,650,0.581829,0.540000,4.182937
12,250,750,0.576744,0.535354,4.139065
19,300,750,0.576697,0.535354,4.134353
18,300,700,0.580171,0.535354,4.481782
16,300,600,0.583934,0.535354,4.858055
15,300,550,0.586393,0.535354,5.103933


In [23]:
results_df_precision_sorted2 = results_df_precision_sorted[:30].sort_values(by='testing delta')
results_df_precision_sorted2

,n_estimators,min_samples_split,training precision,testing precision,testing delta
5,200,750,0.577165,0.540816,3.634855
20,300,800,0.574738,0.535354,3.938458
19,300,750,0.576697,0.535354,4.134353
12,250,750,0.576744,0.535354,4.139065
17,300,650,0.581829,0.540000,4.182937
14,300,500,0.588898,0.545455,4.344362
13,250,800,0.574070,0.530612,4.345811
6,200,800,0.575202,0.530612,4.458937
18,300,700,0.580171,0.535354,4.481782
7,250,500,0.590672,0.545455,4.521795


In [12]:
fit_train_score(df,250,750)

{'n_estimators': 250,
 'min_samples_split': 750,
 'training precision': 0.5764729620661824,
 'testing precision': 0.5353535353535354}

In [24]:
fit_train_score(df,200,900)

{'n_estimators': 200,
 'min_samples_split': 900,
 'training precision': 0.5718281718281718,
 'testing precision': 0.5360824742268041}

In [25]:
fit_train_score(df,200,1000)

{'n_estimators': 200,
 'min_samples_split': 1000,
 'training precision': 0.5687053216838761,
 'testing precision': 0.5306122448979592}

In [28]:
fit_train_score(df,200,800)

{'n_estimators': 200,
 'min_samples_split': 800,
 'training precision': 0.5752016129032258,
 'testing precision': 0.5306122448979592}

In [ ]:
############################################################################
############################################################################
############################################################################

In [27]:
df = get_data('SPY',1999,1,1)

[*********************100%***********************]  1 of 1 completed


In [28]:
df['Range'] = abs(df['High']-df['Low'])
df['RSI 14'] = ta.rsi(df['Close'], 14)
df['SMA 20'] = ta.sma(df['Close'], 20)
df['SMA 50'] = ta.sma(df['Close'], 50)
df['SMA 200'] = ta.sma(df['Close'], 200)
df['EMA 12'] = ta.ema(df['Close'], 12)
df['EMA 26'] = ta.ema(df['Close'], 26)
df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'] )

# create 'Tomorrow' column
# it's the 'Close' value from the previous day
# will be used to create a target
df['Tomorrow'] = df['Close'].shift(-1)
df['Target'] = (df['Tomorrow'] > df['Close']).astype(int)
df = df.copy().loc['2000-01-01':]
df.drop('Adj Close', axis=1, inplace=True)
df.drop(df.tail(1).index, inplace=True)

In [136]:
# foreign market data

In [29]:
# British market data
UK_df = get_data('^FTSE',1999,1,1)

[*********************100%***********************]  1 of 1 completed


In [30]:
# Chinese market data
China_df = get_data('000001.SS',1999,1,1)

[*********************100%***********************]  1 of 1 completed


In [31]:
# German market data
Germany_df = get_data('^GDAXI',1999,1,1)

[*********************100%***********************]  1 of 1 completed


In [32]:
# Japanese market data
Japan_df = get_data('^N225',1999,1,1)

[*********************100%***********************]  1 of 1 completed


In [33]:
# gold
gold_df = get_data('GC=F',1999,1,1)

[*********************100%***********************]  1 of 1 completed


In [142]:
# clean foreign market data
# some have empty volume columns for early 2000's

In [34]:
# drop volume from foreign data dfs
#rename columns
# remove first year of dat

UK_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
UK_df.rename(columns={
    'Open':'UK Open',
    'High':'UK High',
    'Low':'UK Low',
    'Close':'UK Close'
},inplace=True)
UK_df.loc['2000-01-01':]

China_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
China_df.rename(columns={
    'Open':'China Open',
    'High':'China High',
    'Low':'China Low',
    'Close':'China Close'
},inplace=True)
China_df.loc['2000-01-01':]

Germany_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
Germany_df.rename(columns={
    'Open':'Germany Open',
    'High':'Germany High',
    'Low':'Germany Low',
    'Close':'Germany Close'
},inplace=True)
Germany_df.loc['2000-01-01':]

Japan_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
Japan_df.rename(columns={
    'Open':'Japan Open',
    'High':'Japan High',
    'Low':'Japan Low',
    'Close':'Japan Close'
},inplace=True)
Japan_df.loc['2000-01-01':]

gold_df.drop(['Volume', 'Adj Close'], axis=1, inplace=True)
gold_df.rename(columns={
    'Open':'Gold Open',
    'High':'Gold High',
    'Low':'Gold Low',
    'Close':'Gold Close'
},inplace=True)
gold_df.loc['2000-01-01':]

,Gold Open,Gold High,Gold Low,Gold Close
Date,,,,
2000-08-30,273.899994,273.899994,273.899994,273.899994
2000-08-31,274.799988,278.299988,274.799988,278.299988
2000-09-01,277.000000,277.000000,277.000000,277.000000
2000-09-05,275.799988,275.799988,275.799988,275.799988
2000-09-06,274.200012,274.200012,274.200012,274.200012
...,...,...,...,...
2023-08-21,1893.599976,1893.599976,1893.300049,1893.300049
2023-08-22,1894.599976,1896.800049,1892.800049,1896.400024
2023-08-23,1909.599976,1918.500000,1909.599976,1918.500000


In [144]:
UK_df

,UK Open,UK High,UK Low,UK Close
Date,,,,
1999-01-04,5909.399902,5916.899902,5811.299805,5879.399902
1999-01-05,5882.299805,5980.500000,5875.799805,5958.200195
1999-01-06,5968.899902,6157.399902,5968.899902,6148.799805
1999-01-07,6145.899902,6153.700195,6042.500000,6101.200195
1999-01-08,6115.399902,6195.600098,6114.799805,6147.200195
...,...,...,...,...
2023-08-18,7310.200195,7310.200195,7215.799805,7262.399902
2023-08-21,7262.399902,7316.000000,7250.899902,7257.799805
2023-08-22,7257.799805,7310.500000,7257.600098,7270.799805


In [35]:
dfs = [UK_df, China_df, Germany_df, Japan_df, gold_df, df]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='inner'), dfs)
df_merged

,UK Open,UK High,UK Low,UK Close,China Open,China High,China Low,China Close,Germany Open,Germany High,...,Range,RSI 14,SMA 20,SMA 50,SMA 200,EMA 12,EMA 26,ATR,Tomorrow,Target
Date,,,,,,,,,,,,,,,,,,,,,
2000-08-30,6586.299805,6624.899902,6586.100098,6615.100098,2107.019043,2108.096924,2075.159912,2076.143066,7292.459961,7292.459961,...,1.156250,55.725080,149.267188,147.763125,144.410859,150.345132,149.168169,1.661060,152.343750,1
2000-08-31,6615.100098,6675.700195,6585.200195,6672.700195,2062.834961,2064.760010,2019.447021,2021.198975,7191.000000,7251.310059,...,2.187500,62.639763,149.604688,147.852500,144.466328,150.652612,149.403397,1.738841,152.500000,1
2000-09-01,6672.700195,6838.600098,6672.700195,6795.000000,2009.151001,2026.350952,1997.717041,1999.859009,7221.410156,7397.120117,...,1.593750,63.124303,149.910937,147.990000,144.520703,150.936826,149.632775,1.728478,151.281250,0
2000-09-05,6798.100098,6809.500000,6737.600098,6752.500000,2015.062012,2020.847046,1959.592041,1963.791992,7448.080078,7456.709961,...,1.390625,56.922949,150.068750,148.128125,144.563984,150.989814,149.754884,1.725551,149.562500,0
2000-09-06,6752.500000,6766.000000,6688.399902,6694.700195,1955.499023,1964.265015,1923.598022,1925.906982,7397.049805,7434.020020,...,2.421875,49.532617,150.112500,148.194687,144.599297,150.770227,149.740634,1.775288,150.843750,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-18,7310.200195,7310.200195,7215.799805,7262.399902,3165.100098,3179.120117,3131.949951,3131.949951,15597.120117,15612.879883,...,4.559998,35.283750,448.740503,444.149401,412.000000,443.726081,445.381265,4.519322,439.339996,1
2023-08-21,7262.399902,7316.000000,7250.899902,7257.799805,3125.989990,3134.959961,3092.979980,3092.979980,15573.950195,15710.209961,...,4.789978,40.887683,447.997502,444.353600,412.274100,443.051299,444.933763,4.538655,438.149994,0
2023-08-22,7257.799805,7310.500000,7257.600098,7270.799805,3103.780029,3126.780029,3074.530029,3120.330078,15684.639648,15799.339844,...,3.609985,39.350095,447.133002,444.518600,412.590500,442.297252,444.431262,4.472321,443.029999,1


In [146]:
features = get_features(df_merged)

In [ ]:
#############################################################################
# promising models  below

In [149]:
fit_train_score(df_merged,200,750)

{'n_estimators': 200,
 'min_samples_split': 750,
 'training precision': 0.5877170903599295,
 'testing precision': 0.5833333333333334}

In [150]:
fit_train_score(df_merged,300,750)

{'n_estimators': 300,
 'min_samples_split': 750,
 'training precision': 0.5880272796160647,
 'testing precision': 0.5833333333333334}

In [151]:
fit_train_score(df_merged,500, 750)

{'n_estimators': 500,
 'min_samples_split': 750,
 'training precision': 0.5874371859296482,
 'testing precision': 0.5833333333333334}

In [152]:
fit_train_score(df_merged,100,750)


{'n_estimators': 100,
 'min_samples_split': 750,
 'training precision': 0.5872286079182631,
 'testing precision': 0.5945945945945946}

In [155]:
fit_train_score(df_merged,150,750)

{'n_estimators': 150,
 'min_samples_split': 750,
 'training precision': 0.5884586180713743,
 'testing precision': 0.5833333333333334}

In [156]:
fit_train_score(df_merged,75,750)

{'n_estimators': 75,
 'min_samples_split': 750,
 'training precision': 0.5871814282109513,
 'testing precision': 0.5714285714285714}

In [ ]:
#######################################################################

In [44]:
# change the split method
# include more data in the testing set
# testing data now has 1000 rows
# write new fit_train_score function
def fit_train_score_2(df, n_est, min_split):    
    # n_estimators = number of decision trees
    # min_samples_split = higher it is set, the less accurtate it is, the less it will overfit
    model = RandomForestClassifier(n_estimators=n_est, min_samples_split=min_split, random_state=1)

    train = df.iloc[:-1000]
    test = df.iloc[-1000:]

    # fit and train model
    model.fit(train[features], train['Target'])
    predictions = model.predict(test[features])
    #predictions_series = pd.Series(predictions, index=test.index)

    # precision of training data
    predictions_training = model.predict(train[features])
    
    test_dict = {
        'n_estimators':n_est,
        'min_samples_split':min_split,
        'training precision':precision_score(train['Target'], predictions_training),
        'testing precision':precision_score(test['Target'], predictions)
                }
    
    return test_dict

In [36]:
features = get_features(df_merged)

In [161]:
fit_train_score_2(df_merged,100,750)

{'n_estimators': 100,
 'min_samples_split': 750,
 'training precision': 0.5824476161554814,
 'testing precision': 0.5603448275862069}

In [15]:
# use gridsearchCV to find optimal parameters
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

model = RandomForestClassifier(random_state=1)

train = df_merged.iloc[:-1000]
test = df_merged.iloc[-1000:]

CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
CV_rfc.fit(train[features], train['Target'])

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 500]})

In [16]:
CV_rfc.best_params_


{'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'auto',
 'n_estimators': 200}

In [20]:
model = RandomForestClassifier(random_state=1,criterion='entropy', max_depth=4, max_features = 'auto', n_estimators=200)

train = df_merged.iloc[:-1000]
test = df_merged.iloc[-1000:]

model.fit(train[features], train['Target'])
predictions = model.predict(test[features])
predictions_training = model.predict(train[features])


test_dict = {
    'training precision':precision_score(train['Target'], predictions_training),
    'testing precision':precision_score(test['Target'], predictions)
            }
print(test_dict)

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'training precision': 0.5782485875706215, 'testing precision': 0.5548387096774193}


In [21]:
model = RandomForestClassifier(random_state=1,criterion='entropy', max_depth=4, max_features = 'auto', n_estimators=200,min_samples_split=750)

train = df_merged.iloc[:-1000]
test = df_merged.iloc[-1000:]

model.fit(train[features], train['Target'])
predictions = model.predict(test[features])
predictions_training = model.predict(train[features])


test_dict = {
    'training precision':precision_score(train['Target'], predictions_training),
    'testing precision':precision_score(test['Target'], predictions)
            }
print(test_dict)

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'training precision': 0.5576976809164571, 'testing precision': 0.5571428571428572}


In [22]:
# removed max_features hyperparameter
model = RandomForestClassifier(random_state=1,criterion='entropy', max_depth=4, n_estimators=200,min_samples_split=750)

train = df_merged.iloc[:-1000]
test = df_merged.iloc[-1000:]

model.fit(train[features], train['Target'])
predictions = model.predict(test[features])
predictions_training = model.predict(train[features])


test_dict = {
    'training precision':precision_score(train['Target'], predictions_training),
    'testing precision':precision_score(test['Target'], predictions)
            }
print(test_dict)

{'training precision': 0.5576976809164571, 'testing precision': 0.5571428571428572}


In [23]:
model = RandomForestClassifier(random_state=1,criterion='entropy', max_depth=4, max_features = 'auto', n_estimators=200,min_samples_split=800)

train = df_merged.iloc[:-1000]
test = df_merged.iloc[-1000:]

model.fit(train[features], train['Target'])
predictions = model.predict(test[features])
predictions_training = model.predict(train[features])


test_dict = {
    'training precision':precision_score(train['Target'], predictions_training),
    'testing precision':precision_score(test['Target'], predictions)
            }
print(test_dict)

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'training precision': 0.556732645664901, 'testing precision': 0.5523560209424084}


In [40]:
# optimizing hyperparameters for random forest model
# restricting n_estimators to 200 & 250
# include min_samples_split
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [200, 250],
    'min_samples_split': [400, 450, 500, 550, 600,650,700],
    'max_features': ['auto'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['entropy']
}

model = RandomForestClassifier(random_state=1)

train = df_merged.iloc[:-1000]
test = df_merged.iloc[-1000:]

CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
CV_rfc.fit(train[features], train['Target'])

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

...

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
             param_grid={'criterion': ['entropy'], 'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto'],
                         'min_samples_split': [400, 450, 500, 550, 600, 650,
                                               700],
                         'n_estimators': [200, 250]})

In [41]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'auto',
 'min_samples_split': 650,
 'n_estimators': 200}

In [42]:
model = RandomForestClassifier(random_state=1,criterion='entropy', max_depth=4, max_features = 'auto', n_estimators=200,min_samples_split=650)

train = df_merged.iloc[:-1000]
test = df_merged.iloc[-1000:]

model.fit(train[features], train['Target'])
predictions = model.predict(test[features])
predictions_training = model.predict(train[features])


test_dict = {
    'training precision':precision_score(train['Target'], predictions_training),
    'testing precision':precision_score(test['Target'], predictions)
            }
print(test_dict)

C:\Users\13148\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'training precision': 0.5591277606933184, 'testing precision': 0.5609756097560976}


In [ ]:
# results from optimized models are still worse than models highlighted above
# however, models above were trained with a different split (only 100 rows in test set)

# try running models from above with different split (larger test set)

In [49]:
fit_train_score_2(df_merged,100,750)


{'n_estimators': 100,
 'min_samples_split': 750,
 'training precision': 0.5813672111312764,
 'testing precision': 0.544}

In [47]:
fit_train_score_2(df_merged,500,750)


{'n_estimators': 500,
 'min_samples_split': 750,
 'training precision': 0.584882657726303,
 'testing precision': 0.625}

In [50]:
fit_train_score_2(df_merged,200,750)


{'n_estimators': 200,
 'min_samples_split': 750,
 'training precision': 0.5841704718417047,
 'testing precision': 0.5684210526315789}

In [51]:
fit_train_score_2(df_merged,300,750)


{'n_estimators': 300,
 'min_samples_split': 750,
 'training precision': 0.5843446601941747,
 'testing precision': 0.6}

In [52]:
fit_train_score_2(df_merged,150,750)


{'n_estimators': 150,
 'min_samples_split': 750,
 'training precision': 0.581784952787085,
 'testing precision': 0.5555555555555556}

In [53]:
fit_train_score_2(df_merged,75,750)


{'n_estimators': 75,
 'min_samples_split': 750,
 'training precision': 0.5820985787723012,
 'testing precision': 0.5638297872340425}

In [ ]:
###############

In [60]:
# change n_estimators to 200 as recommended by grid search CV


In [61]:
fit_train_score_2(df_merged,200,200)


{'n_estimators': 200,
 'min_samples_split': 200,
 'training precision': 0.6612173913043479,
 'testing precision': 0.7222222222222222}

In [70]:
fit_train_score_2(df_merged,200,210)


{'n_estimators': 200,
 'min_samples_split': 210,
 'training precision': 0.6588153792864565,
 'testing precision': 0.8125}

In [62]:
fit_train_score_2(df_merged,200,250)


{'n_estimators': 200,
 'min_samples_split': 250,
 'training precision': 0.6448882870683819,
 'testing precision': 0.6666666666666666}

In [67]:
fit_train_score_2(df_merged,200,260)


{'n_estimators': 200,
 'min_samples_split': 260,
 'training precision': 0.6432689047134622,
 'testing precision': 0.7272727272727273}

In [68]:
fit_train_score_2(df_merged,200,275)


{'n_estimators': 200,
 'min_samples_split': 275,
 'training precision': 0.6402829235432805,
 'testing precision': 0.6818181818181818}

In [63]:
fit_train_score_2(df_merged,200,300)


{'n_estimators': 200,
 'min_samples_split': 300,
 'training precision': 0.633399866932801,
 'testing precision': 0.6956521739130435}

In [66]:
fit_train_score_2(df_merged,200,350)


{'n_estimators': 200,
 'min_samples_split': 350,
 'training precision': 0.6234791187109503,
 'testing precision': 0.75}

In [71]:
fit_train_score_2(df_merged,200,650)


{'n_estimators': 200,
 'min_samples_split': 650,
 'training precision': 0.592398022249691,
 'testing precision': 0.609375}

In [74]:
fit_train_score_2(df_merged,200,625)


{'n_estimators': 200,
 'min_samples_split': 625,
 'training precision': 0.592695759826679,
 'testing precision': 0.6119402985074627}

In [72]:
fit_train_score_2(df_merged,200,600)


{'n_estimators': 200,
 'min_samples_split': 600,
 'training precision': 0.5961358678716111,
 'testing precision': 0.6111111111111112}

In [73]:
fit_train_score_2(df_merged,200,700)


{'n_estimators': 200,
 'min_samples_split': 700,
 'training precision': 0.5869365225390984,
 'testing precision': 0.5952380952380952}